In [1]:
# Import third party packages
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
import trackpy
from iris.time import PartialDateTime
import cartopy.crs as ccrs
import xarray as xr
import netCDF4 as nc
import scipy
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure
import tobac #tobac package cloned from https://github.com/tobac-project/tobac.git

In [2]:
# Read in the yearly initial tracks file for each year
file_2005 = '/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005.h5'

In [3]:
tracks_2005 = pd.read_hdf(file_2005, 'table')

In [4]:
tracks_2005

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,cell,time_cell
0,0,9,179.648443,518.713643,3617,240,1,2005-01-01 00:30:00,2005-01-01 00:30:00,-32.764288,295.967957,1,0 days 00:00:00
1,0,26,319.360864,876.726404,7359,240,2,2005-01-01 00:30:00,2005-01-01 00:30:00,-27.105936,310.467466,2,0 days 00:00:00
2,0,57,465.549812,1014.466367,5384,240,3,2005-01-01 00:30:00,2005-01-01 00:30:00,-21.185283,316.045939,-1,NaT
3,0,73,763.319602,705.760312,47759,240,4,2005-01-01 00:30:00,2005-01-01 00:30:00,-9.125606,303.543333,-1,NaT
4,0,86,577.470368,717.240701,2132,240,5,2005-01-01 00:30:00,2005-01-01 00:30:00,-16.652501,304.008313,-1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104164,8759,192,780.546661,938.590110,7445,240,104165,2005-12-31 23:30:00,2005-12-31 23:30:00,-8.427910,312.972951,-1,NaT
104165,8759,202,773.219319,219.806523,2686,240,104166,2005-12-31 23:30:00,2005-12-31 23:30:00,-8.724668,283.862221,-1,NaT
104166,8759,216,846.627296,635.279440,13115,240,104167,2005-12-31 23:30:00,2005-12-31 23:30:00,-5.751644,300.688868,-1,NaT
104167,8759,265,960.267992,1283.290530,5481,240,104168,2005-12-31 23:30:00,2005-12-31 23:30:00,-1.149196,326.933305,23469,0 days 07:00:00


In [5]:
(np.unique(tracks_2005.cell.values).shape[0])

4544

In [6]:
# Sort dataframe into date order by first converting timestr to datetime
tracks_2005["timestr"] = tracks_2005["timestr"].astype('datetime64[ns]')


In [7]:
# jan = tracks_2005[tracks_2005['timestr'].dt.month==1]
# feb = tracks_2005[tracks_2005['timestr'].dt.month==2]
# mar = tracks_2005[tracks_2005['timestr'].dt.month==3]
# apr = tracks_2005[tracks_2005['timestr'].dt.month==4]
# may = tracks_2005[tracks_2005['timestr'].dt.month==5]
# jun = tracks_2005[tracks_2005['timestr'].dt.month==6]
# jul = tracks_2005[tracks_2005['timestr'].dt.month==7]


In [8]:
jan_2005 = pd.DataFrame()
feb_2005 = pd.DataFrame()
mar_2005 = pd.DataFrame()
apr_2005 = pd.DataFrame()
may_2005 = pd.DataFrame()
jun_2005 = pd.DataFrame()
jul_2005 = pd.DataFrame()
aug_2005 = pd.DataFrame()
sep_2005 = pd.DataFrame()
oct_2005 = pd.DataFrame()
nov_2005 = pd.DataFrame()
dec_2005 = pd.DataFrame()


In [9]:
subset=tracks_2005[tracks_2005.cell > 0]

# for month in month_list:
#     print(month)
#     subset = tracks_2005[tracks_2005['timestr'].dt.month==month]
#     print(subset)
unique_cells = np.unique(subset.cell.values)
print(np.shape(unique_cells))
for cell in unique_cells:
    cell_subset = subset[subset.cell == cell]
    if int(cell_subset.head(1).timestr.dt.month) == 1:
        jan_2005 = pd.concat([jan_2005, cell_subset])
    if int(cell_subset.head(1).timestr.dt.month) == 2:
        feb_2005 = pd.concat([feb_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 3:
        mar_2005 = pd.concat([mar_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 4:
        apr_2005 = pd.concat([apr_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 5:
        may_2005 = pd.concat([may_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 6:
        jun_2005 = pd.concat([jun_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 7:
        jul_2005 = pd.concat([jul_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 8:
        aug_2005 = pd.concat([aug_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 9:
        sep_2005 = pd.concat([sep_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 10:
        oct_2005 = pd.concat([oct_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 11:
        nov_2005 = pd.concat([nov_2005, cell_subset])
    elif int(cell_subset.head(1).timestr.dt.month) == 12:
        dec_2005 = pd.concat([dec_2005, cell_subset])
         

(4543,)


In [10]:
print(np.unique(jan_2005.cell.values).shape[0])
print(np.unique(feb_2005.cell.values).shape[0])
print(np.unique(mar_2005.cell.values).shape[0])
print(np.unique(apr_2005.cell.values).shape[0])
print(np.unique(may_2005.cell.values).shape[0])
print(np.unique(jun_2005.cell.values).shape[0])
print(np.unique(jul_2005.cell.values).shape[0])
print(np.unique(aug_2005.cell.values).shape[0])
print(np.unique(sep_2005.cell.values).shape[0])
print(np.unique(oct_2005.cell.values).shape[0])
print(np.unique(oct_2005.cell.values).shape[0])
print(np.unique(dec_2005.cell.values).shape[0])

395
372
405
406
413
373
336
357
301
396
396
408


In [11]:
jan_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_01.h5', 'table')
feb_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_02.h5', 'table')
mar_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_03.h5', 'table')
apr_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_04.h5', 'table')
may_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_05.h5', 'table')
jun_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_06.h5', 'table')
jul_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_07.h5', 'table')
aug_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_08.h5', 'table')
sep_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_09.h5', 'table')
oct_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_10.h5', 'table')
nov_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_11.h5', 'table')
dec_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_12.h5', 'table')












/var/tmp/ipykernel_83325/914471666.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['time'], dtype='object')]

  jan_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_01.h5', 'table')
/var/tmp/ipykernel_83325/914471666.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['time'], dtype='object')]

  feb_2005.to_hdf('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/tracking/tracks_2005_02.h5', 'table')
/var/tmp/ipykernel_83325/914471666.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['time'], dtype='object')]

  mar_2005.to_hdf('/da